# Mapping of perturbed samples

In [2]:
import scanpy as sc
import numpy as np
import scanpy.external as sce
import pandas as pd
import scvelo as scv
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import sys
import symphonypy as sp

# Figures aesthetics
sc.set_figure_params(dpi=150)

# Pathes
DATA_PATH = "/home/sergey/data/clonal_project"
HOME_PATH = "/home/sergey/projects/clonal_project"

# Additional functions
exec(open(f"{HOME_PATH}/tools/tools.py").read())
sns.set_style("ticks")

g2m_genes = list(pd.read_csv("/home/sergey/data/additional/cell_cycle_genes/G2M_phase.txt", names=["gene"]).gene)
g2m_genes = [gene[0].upper() + gene[1:].lower() for gene in g2m_genes]

s_genes = list(pd.read_csv("/home/sergey/data/additional/cell_cycle_genes/S_phase.txt", names=["gene"]).gene)
s_genes = [gene[0].upper() + gene[1:].lower() for gene in s_genes]

In [3]:
injections = ["injection8", "injection9", "injection10", "injection18"]

adatas = []
for injection in injections:
    adata = sc.read_h5ad(f"{DATA_PATH}/anndatas_counts/{injection}.h5ad")
    if "E8.5:clones" in adata.obs:
        adata.obs["E8.5:clones"] = adata.obs["E8.5:clones"].fillna("NA")
    if "E7.5:clones" in adata.obs:
        adata.obs["E7.5:clones"] = adata.obs["E7.5:clones"].fillna("NA")
    adata = adata[:, ~np.isin(adata.var_names, ["GFPbc", "TOMbc"])]
    adata.layers["counts"] = adata.X.copy()
    adata.var.mt = adata.var.mt.astype(bool)
    adatas.append(adata)
    
adata = adatas[0].concatenate(
    adatas[1:],
    join="outer",
    batch_key=None,
    uns_merge="first",
    index_unique=None,
    fill_value=0,
)
for var_column in adata.var:
    del adata.var[var_column]

In [4]:
adata.write_h5ad("/home/sergey/data/clonal_project/anndatas_counts/reference_perturbations_E13_noEtOH.h5ad")

## Trunk

In [5]:
face_samples = ["H2", "F1", "N1"]
adata = adata[~adata.obs.sample_id.isin(face_samples)]

In [6]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

adata_cycling = adata[adata.obs.phase != "G1"]
adata_noncycling = adata[adata.obs.phase == "G1"]

In [7]:
adata_trunk = sc.read_h5ad(f"{DATA_PATH}/anndatas/trunk_E13_reference.h5ad")
adata_trunk_m = sc.read_h5ad(f"{DATA_PATH}/anndatas/mesenchyme_trunk_E13_reference.h5ad")
adata_trunk_n = sc.read_h5ad(f"{DATA_PATH}/anndatas/neurons_glia_trunk_E13_reference.h5ad")
adata_trunk_o = sc.read_h5ad(f"{DATA_PATH}/anndatas/other_cells_trunk_E13_reference.h5ad")
adata_trunk_g = sc.read_h5ad(f"{DATA_PATH}/anndatas/glia_trunk_E13_reference.h5ad")

In [8]:
sp.tl.map_embedding(adata_noncycling, adata_trunk, key="sample_id")
sp.tl.map_embedding(adata_cycling, adata_trunk, key="sample_id")

In [9]:
sp.tl.per_cell_confidence(adata_noncycling, adata_trunk)
sp.tl.per_cell_confidence(adata_cycling, adata_trunk)

In [10]:
sp.tl.transfer_labels_kNN(adata_noncycling, adata_trunk, "celltype_l0")
sp.tl.transfer_labels_kNN(adata_cycling, adata_trunk, "celltype_l0")

### Mesenchyme

In [11]:
adata_noncycling_m = adata_noncycling[adata_noncycling.obs.celltype_l0 == "Mesenchyme"]
adata_cycling_m = adata_cycling[adata_cycling.obs.celltype_l0 == "Mesenchyme"]

sp.tl.map_embedding(adata_noncycling_m, adata_trunk_m, key="sample_id")
sp.tl.map_embedding(adata_cycling_m, adata_trunk_m, key="sample_id")

sp.tl.per_cell_confidence(adata_noncycling_m, adata_trunk_m)
sp.tl.per_cell_confidence(adata_cycling_m, adata_trunk_m)

sp.tl.transfer_labels_kNN(adata_noncycling_m, adata_trunk_m, ["celltype_l2", "celltype_l3"])
sp.tl.transfer_labels_kNN(adata_cycling_m, adata_trunk_m, ["celltype_l2", "celltype_l3"])

sp.tl.ingest(adata_noncycling_m, adata_trunk_m, use_rep="X_pca_harmony")
sp.tl.ingest(adata_cycling_m, adata_trunk_m, use_rep="X_pca_harmony")

2024-04-05 22:32:06.625115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-05 22:32:06.678817: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-05 22:32:08.053706: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-05 22:32:08.053852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-04-05 22:32:08.053866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Ca

### Other cells

In [12]:
adata_noncycling_o = adata_noncycling[adata_noncycling.obs.celltype_l0 == "Other"]
adata_cycling_o = adata_cycling[adata_cycling.obs.celltype_l0 == "Other"]

sp.tl.map_embedding(adata_noncycling_o, adata_trunk_o, key="sample_id")
sp.tl.map_embedding(adata_cycling_o, adata_trunk_o, key="sample_id")

sp.tl.per_cell_confidence(adata_noncycling_o, adata_trunk_o)
sp.tl.per_cell_confidence(adata_cycling_o, adata_trunk_o)

sp.tl.transfer_labels_kNN(adata_noncycling_o, adata_trunk_o, ["celltype_l2", "celltype_l3"])
sp.tl.transfer_labels_kNN(adata_cycling_o, adata_trunk_o, ["celltype_l2", "celltype_l3"])

sp.tl.ingest(adata_noncycling_o, adata_trunk_o, use_rep="X_pca_harmony")
sp.tl.ingest(adata_cycling_o, adata_trunk_o, use_rep="X_pca_harmony")

### Neurons

In [13]:
adata_noncycling_n = adata_noncycling[adata_noncycling.obs.celltype_l0.isin(["Neurons", "Glia"])]
adata_cycling_n = adata_cycling[adata_cycling.obs.celltype_l0.isin(["Neurons", "Glia"])]

sp.tl.map_embedding(adata_noncycling_n, adata_trunk_n, key="sample_id")
sp.tl.map_embedding(adata_cycling_n, adata_trunk_n, key="sample_id")

sp.tl.per_cell_confidence(adata_noncycling_n, adata_trunk_n)
sp.tl.per_cell_confidence(adata_cycling_n, adata_trunk_n)

sp.tl.transfer_labels_kNN(adata_noncycling_n, adata_trunk_n, ["celltype_l2", "celltype_l3"])
sp.tl.transfer_labels_kNN(adata_cycling_n, adata_trunk_n, ["celltype_l2", "celltype_l3"])

sp.tl.ingest(adata_noncycling_n, adata_trunk_n, use_rep="X_pca_harmony")
sp.tl.ingest(adata_cycling_n, adata_trunk_n, use_rep="X_pca_harmony")

### Glia

In [14]:
groups = ["Glia", "Autonomic sympathetic", "Chromaffin cells", "Autonomic parasympathetic"]
adata_noncycling_g = adata_noncycling_n[adata_noncycling_n.obs.celltype_l2.isin(groups)]
adata_cycling_g = adata_cycling_n[adata_cycling_n.obs.celltype_l2.isin(groups)]

sp.tl.map_embedding(adata_noncycling_g, adata_trunk_g, key="sample_id")
sp.tl.map_embedding(adata_cycling_g, adata_trunk_g, key="sample_id")

sp.tl.per_cell_confidence(adata_noncycling_g, adata_trunk_g)
sp.tl.per_cell_confidence(adata_cycling_g, adata_trunk_g)

sp.tl.transfer_labels_kNN(adata_noncycling_g, adata_trunk_g, ["celltype_l2", "celltype_l3"])
sp.tl.transfer_labels_kNN(adata_cycling_g, adata_trunk_g, ["celltype_l2", "celltype_l3"])

sp.tl.ingest(adata_noncycling_g, adata_trunk_g, use_rep="X_pca_harmony")
sp.tl.ingest(adata_cycling_g, adata_trunk_g, use_rep="X_pca_harmony")

### Finalising the results

In [15]:
sp.tl.ingest(adata_noncycling, adata_trunk, use_rep="X_pca_harmony")
sp.tl.ingest(adata_cycling, adata_trunk, use_rep="X_pca_harmony")

In [16]:
adata_full = adata_cycling.concatenate(adata_noncycling, index_unique=None, batch_key=None, uns_merge="first")

adata_full.obs["celltype_l2"] = np.nan
adata_full.obs["celltype_l3"] = np.nan

for adata in [
    adata_cycling_n, adata_cycling_g, adata_cycling_m, adata_cycling_o,
    adata_noncycling_n, adata_noncycling_g, adata_noncycling_m, adata_noncycling_o,
]:
    adata_full.obs["celltype_l2"].loc[adata.obs_names] = adata.obs["celltype_l2"]
    adata_full.obs["celltype_l3"].loc[adata.obs_names] = adata.obs["celltype_l3"]

### Saving the results

In [17]:
!mkdir $DATA_PATH/anndatas/perturbations_noEtOH

In [18]:
adata_full_n = adata_cycling_n.concatenate(adata_noncycling_n, index_unique=None, batch_key=None, uns_merge="first")
adata_full_o = adata_cycling_o.concatenate(adata_noncycling_o, index_unique=None, batch_key=None, uns_merge="first")
adata_full_g = adata_cycling_g.concatenate(adata_noncycling_g, index_unique=None, batch_key=None, uns_merge="first")
adata_full_m = adata_cycling_m.concatenate(adata_noncycling_m, index_unique=None, batch_key=None, uns_merge="first")

adata_full_n.write_h5ad(f"{DATA_PATH}/anndatas/perturbations_noEtOH/trunk_E13_neurons_and_glia.h5ad")
adata_full_o.write_h5ad(f"{DATA_PATH}/anndatas/perturbations_noEtOH/trunk_E13_other.h5ad")
adata_full_g.write_h5ad(f"{DATA_PATH}/anndatas/perturbations_noEtOH/trunk_E13_glia.h5ad")
adata_full_m.write_h5ad(f"{DATA_PATH}/anndatas/perturbations_noEtOH/trunk_E13_mesenchyme.h5ad")

adata_full.write_h5ad(f"{DATA_PATH}/anndatas/perturbations_noEtOH/trunk_E13.h5ad")

In [19]:
!mkdir $DATA_PATH/clonal_composition/perturbations_noEtOH

In [20]:
for timepoint in ["E7.5", "E8.5"]:
    adata_full.obs.groupby([f"{timepoint}:clones", "celltype_l3"]).size().unstack().to_csv(f"{DATA_PATH}/clonal_composition/perturbations_noEtOH/trunk_{timepoint}_l3.csv")
    adata_full.obs.groupby([f"{timepoint}:clones", "celltype_l2"]).size().unstack().to_csv(f"{DATA_PATH}/clonal_composition/perturbations_noEtOH/trunk_{timepoint}_l2.csv")
    adata_full.obs.groupby([f"{timepoint}:clones", "sample_id"]).size().unstack().to_csv(f"{DATA_PATH}/clonal_composition/perturbations_noEtOH/trunk_{timepoint}_sample_id.csv")
    pd.DataFrame(
        [(i[0], i[1]) for i in set(zip(adata_full.obs[f"{timepoint}:clones"], adata_full.obs["CRISPR:predicted"])) if str(i[0]) != "nan"],
        columns=[f"{timepoint}:clones", "CRISPR:predicted"],
    ).fillna("NA").to_csv(f"{DATA_PATH}/clonal_composition/perturbations_noEtOH/trunk_{timepoint}_gRNA.csv")

## Face

In [21]:
adata = sc.read_h5ad("/home/sergey/data/clonal_project/anndatas_counts/reference_perturbations_E13_noEtOH.h5ad")

In [22]:
face_samples = ["H2", "F1", "N1"]
adata = adata[adata.obs.sample_id.isin(face_samples)]

In [23]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

adata_cycling = adata[adata.obs.phase != "G1"]
adata_noncycling = adata[adata.obs.phase == "G1"]

In [24]:
adata_face = sc.read_h5ad(f"{DATA_PATH}/anndatas/face_E13_reference.h5ad")
adata_face_m = sc.read_h5ad(f"{DATA_PATH}/anndatas/mesenchyme_face_E13_reference.h5ad")
adata_face_n = sc.read_h5ad(f"{DATA_PATH}/anndatas/neurons_glia_face_E13_reference.h5ad")
adata_face_o = sc.read_h5ad(f"{DATA_PATH}/anndatas/other_cells_face_E13_reference.h5ad")

In [25]:
sp.tl.map_embedding(adata_noncycling, adata_face, key="sample_id")
sp.tl.map_embedding(adata_cycling, adata_face, key="sample_id")

In [26]:
sp.tl.per_cell_confidence(adata_noncycling, adata_face)
sp.tl.per_cell_confidence(adata_cycling, adata_face)

In [27]:
sp.tl.transfer_labels_kNN(adata_noncycling, adata_face, "celltype_l0")
sp.tl.transfer_labels_kNN(adata_cycling, adata_face, "celltype_l0")

### Mesenchyme

In [28]:
adata_noncycling_m = adata_noncycling[adata_noncycling.obs.celltype_l0 == "Mesenchyme"]
adata_cycling_m = adata_cycling[adata_cycling.obs.celltype_l0 == "Mesenchyme"]

sp.tl.map_embedding(adata_noncycling_m, adata_face_m, key="sample_id")
sp.tl.map_embedding(adata_cycling_m, adata_face_m, key="sample_id")

sp.tl.per_cell_confidence(adata_noncycling_m, adata_face_m)
sp.tl.per_cell_confidence(adata_cycling_m, adata_face_m)

sp.tl.transfer_labels_kNN(adata_noncycling_m, adata_face_m, ["celltype_l2", "celltype_l3"])
sp.tl.transfer_labels_kNN(adata_cycling_m, adata_face_m, ["celltype_l2", "celltype_l3"])

sp.tl.ingest(adata_noncycling_m, adata_face_m, use_rep="X_pca_harmony")
sp.tl.ingest(adata_cycling_m, adata_face_m, use_rep="X_pca_harmony")

### Other cells

In [29]:
adata_noncycling_o = adata_noncycling[adata_noncycling.obs.celltype_l0 == "Other"]
adata_cycling_o = adata_cycling[adata_cycling.obs.celltype_l0 == "Other"]

sp.tl.map_embedding(adata_noncycling_o, adata_face_o, key="sample_id")
sp.tl.map_embedding(adata_cycling_o, adata_face_o, key="sample_id")

sp.tl.per_cell_confidence(adata_noncycling_o, adata_face_o)
sp.tl.per_cell_confidence(adata_cycling_o, adata_face_o)

sp.tl.transfer_labels_kNN(adata_noncycling_o, adata_face_o, ["celltype_l2", "celltype_l3"])
sp.tl.transfer_labels_kNN(adata_cycling_o, adata_face_o, ["celltype_l2", "celltype_l3"])

sp.tl.ingest(adata_noncycling_o, adata_face_o, use_rep="X_pca_harmony")
sp.tl.ingest(adata_cycling_o, adata_face_o, use_rep="X_pca_harmony")

### Neurons and glia

In [30]:
adata_noncycling_n = adata_noncycling[adata_noncycling.obs.celltype_l0.isin(["Neurons and glia"])]
adata_cycling_n = adata_cycling[adata_cycling.obs.celltype_l0.isin(["Neurons and glia"])]

sp.tl.map_embedding(adata_noncycling_n, adata_face_n, key="sample_id")
sp.tl.map_embedding(adata_cycling_n, adata_face_n, key="sample_id")

sp.tl.per_cell_confidence(adata_noncycling_n, adata_face_n)
sp.tl.per_cell_confidence(adata_cycling_n, adata_face_n)

sp.tl.transfer_labels_kNN(adata_noncycling_n, adata_face_n, ["celltype_l2", "celltype_l3"])
sp.tl.transfer_labels_kNN(adata_cycling_n, adata_face_n, ["celltype_l2", "celltype_l3"])

sp.tl.ingest(adata_noncycling_n, adata_face_n, use_rep="X_pca_harmony")
sp.tl.ingest(adata_cycling_n, adata_face_n, use_rep="X_pca_harmony")

### Concatenation

In [31]:
sp.tl.ingest(adata_noncycling, adata_face, use_rep="X_pca_harmony")
sp.tl.ingest(adata_cycling, adata_face, use_rep="X_pca_harmony")

In [32]:
adata_full = adata_cycling.concatenate(adata_noncycling, index_unique=None, batch_key=None, uns_merge="first")

adata_full.obs["celltype_l2"] = np.nan
adata_full.obs["celltype_l3"] = np.nan

for adata in [
    adata_cycling_n, adata_cycling_m, adata_cycling_o,
    adata_noncycling_n, adata_noncycling_m, adata_noncycling_o,
]:
    adata_full.obs["celltype_l2"].loc[adata.obs_names] = adata.obs["celltype_l2"]
    adata_full.obs["celltype_l3"].loc[adata.obs_names] = adata.obs["celltype_l3"]

### Saving the results

In [33]:
adata_full_n = adata_cycling_n.concatenate(adata_noncycling_n, index_unique=None, batch_key=None, uns_merge="first")
adata_full_o = adata_cycling_o.concatenate(adata_noncycling_o, index_unique=None, batch_key=None, uns_merge="first")
adata_full_m = adata_cycling_m.concatenate(adata_noncycling_m, index_unique=None, batch_key=None, uns_merge="first")

adata_full_n.write_h5ad(f"{DATA_PATH}/anndatas/perturbations_noEtOH/face_E13_neurons_and_glia.h5ad")
adata_full_o.write_h5ad(f"{DATA_PATH}/anndatas/perturbations_noEtOH/face_E13_other.h5ad")
adata_full_m.write_h5ad(f"{DATA_PATH}/anndatas/perturbations_noEtOH/face_E13_mesenchyme.h5ad")

adata_full.write_h5ad(f"{DATA_PATH}/anndatas/perturbations_noEtOH/face_E13.h5ad")

In [34]:
for timepoint in ["E7.5", "E8.5"]:
    adata_full.obs.groupby([f"{timepoint}:clones", "celltype_l3"]).size().unstack().to_csv(f"{DATA_PATH}/clonal_composition/perturbations_noEtOH/face_{timepoint}_l3.csv")
    adata_full.obs.groupby([f"{timepoint}:clones", "celltype_l2"]).size().unstack().to_csv(f"{DATA_PATH}/clonal_composition/perturbations_noEtOH/face_{timepoint}_l2.csv")
    adata_full.obs.groupby([f"{timepoint}:clones", "sample_id"]).size().unstack().to_csv(f"{DATA_PATH}/clonal_composition/perturbations_noEtOH/face_{timepoint}_sample_id.csv")
    pd.DataFrame(
        [(i[0], i[1]) for i in set(zip(adata_full.obs[f"{timepoint}:clones"], adata_full.obs["CRISPR:predicted"])) if str(i[0]) != "nan"],
        columns=[f"{timepoint}:clones", "CRISPR:predicted"],
    ).fillna("NA").to_csv(f"{DATA_PATH}/clonal_composition/perturbations_noEtOH/face_{timepoint}_gRNA.csv")